In [1]:
# putting the contents of a text file into a panda dataframe

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('death_lessons_dataframe.csv')

In [4]:
df.head()

,person,date_of_report,ref,name_of_deceased,coroner_name,coroner_area,category
0,https:www.judiciary.ukprevention-of-future-dea...,<p>Date of report: 01/07/22</p>,<p>Ref: 2022-0204</p>,<p>Deceased name: Dominic Noble</p>,<p>Coroner name: Kevin McLoughlin</p>,<p>Coroner Area: West Yorkshire (Eastern)</p>,CATEGORY: SUICIDE (FROM 2015) | POLICE RELATED...
1,https:www.judiciary.ukprevention-of-future-dea...,<p>Date of report: 01/07/22</p>,<p>Ref: 2022-0203</p>,<p>Deceased name: Daniel Xavier</p>,<p>Coroner name: Graeme Irvine</p>,<p>Coroner Area: East London</p>,CATEGORY: HOSPITAL DEATH (CLINICAL PROCEDURES ...
2,https:www.judiciary.ukprevention-of-future-dea...,<p>Date of report: 01/07/22</p>,<p>Ref: 2022-0202</p>,<p>Deceased name: Shona Campbell</p>,<p>Coroner name: Nigel Meadows</p>,<p>Coroner Area: Manchester City</p>,CATEGORY: SUICIDE (FROM 2015) | MENTAL HEALTH ...
3,https:www.judiciary.ukprevention-of-future-dea...,<p>Date of report: 29/06/22</p>,<p>Ref: 2022-0201</p>,<p>Deceased name: Paul Meadows</p>,<p>Coroner name: Peter Taheri</p>,<p>Coroner Area: Suffolk</p>,"CATEGORY: ALCOHOL, DRUG AND MEDICATION RELATED..."
4,https:www.judiciary.ukprevention-of-future-dea...,<p>Date of report: 29/06/22</p>,<p>Ref: 2022-0200</p>,<p>Deceased name: Louise Bailey</p>,<p>Coroner name: Jonathan Landau</p>,<p>Coroner Area: Inner South London</p>,CATEGORY: ROAD (HIGHWAYS SAFETY) RELATED DEATH...


In [4]:
files = !ls downloads/

In [5]:
cd downloads

/media/drcjar/Expansion/death-lessons_old/downloads


In [ ]:
text_reports = {}

for f in files:
    try:
        with open(f, 'r') as reader:
            text_reports[f] = reader.read()
    except:
        continue

In [ ]:
df1 = pd.DataFrame.from_dict(text_reports, orient='index', columns=['text'])
df1 = df1.reset_index()
df1.columns = ['filename', 'text']

In [ ]:
# find reference in our dataframe and in our filenames
df['ref'] = df['ref'].str.extract(r'(\d{4}-\d{4})') # extract reference if there is one
df1['ref'] = df1['filename'].str.extract(r'(\d{4}-\d{4})') # extract reference if there is one

In [ ]:
df.ref.isnull().value_counts()

In [ ]:
df1.ref.isnull().value_counts()

In [ ]:
# discard nulls for now
df = df[~df.ref.isnull()]
df1 = df1[~df1.ref.isnull()]

In [ ]:
# merge on ref
df = pd.merge(df,df1, on='ref')

In [ ]:
df.ref.isnull().value_counts()

In [ ]:
df['url'] = 'https://www.judiciary.uk/publications/' + df['person'].astype(str) + '/'

In [ ]:
# clean up data (still contains html tags and duplicates) ? use soup.get_text() earlier
# decide what to display and how
# style with css

In [ ]:
# remove html tags
df['date_of_report'] = df['date_of_report'].str.replace('<[^<]+?>', '')
df['name_of_deceased'] = df['name_of_deceased'].str.replace('<[^<]+?>', '')
df['coroner_name'] = df['coroner_name'].str.replace('<[^<]+?>', '')
df['coroner_area'] = df['coroner_area'].str.replace('<[^<]+?>', '')
# remove duplicates
df = df.drop_duplicates(subset='filename')

In [ ]:
# df.to_json("data.json", lines=True, orient="records`")

In [ ]:
# INVESTIGATION and INQUEST
# CIRCUMSTANCES OF THE DEATH 
# CORONER’S CONCERNS 
# The MATTERS OF CONCERN are as follows
# ACTION SHOULD BE TAKEN 

In [ ]:
df['Coroner\'s report'] = df.text.str.contains("INVESTIGATION and INQUEST")

In [ ]:
df[df['Coroner\'s report']].text.str.extract('(?s)INVESTIGATION\sand\sINQUEST(.*?)CIRCUMSTANCES')

In [ ]:
df['investigation_and_inquest'] = df.text.str.extract('(?s)INVESTIGATION\sand\sINQUEST(.*?)CIRCUMSTANCES')

In [ ]:
df.investigation_and_inquest.isnull().value_counts()

In [ ]:
df['Coroner\'s report'].value_counts()

In [ ]:
# report headers:
# INVESTIGATION and INQUEST
# CIRCUMSTANCES OF THE DEATH 
# CORONER’S CONCERNS 
# The MATTERS OF CONCERN are as follows
# ACTION SHOULD BE TAKEN 

In [ ]:
df['investigation_and_inquest'] = df.text.str.extract('(?s)INVESTIGATION\sand\sINQUEST(.*?)CIRCUMSTANCES')

In [ ]:
df['circumstances_of_death'] = df.text.str.extract('(?s)CIRCUMSTANCES\sOF\sTHE\sDEATH(.*?)CORONER’S\sCONCERNS')

In [ ]:
df['coroners_concerns'] = df.text.str.extract('(?s)CORONER’S\sCONCERNS(.*?)The\sMATTERS\sOF\sCONCERN')

In [ ]:
df['matters_of_concern'] = df.text.str.extract('(?s)The\sMATTERS\sOF\sCONCERN(.*?)ACTION\sSHOULD\sBE\sTAKEN')

In [ ]:
df[df['Coroner\'s report']][['investigation_and_inquest', 'circumstances_of_death', 'coroners_concerns', 'matters_of_concern']].to_csv('extract.csv')

In [ ]:
df[df['Coroner\'s report']][['investigation_and_inquest', 'circumstances_of_death', 'matters_of_concern', 'url']].dropna().to_csv('extract.csv')